In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoAlertPresentException
import time

def handle_alert():
    try:
        WebDriverWait(driver, 2).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        print("alert 감지:", alert.text)
        alert.accept()
        time.sleep(0.5)
    except NoAlertPresentException:
        pass
    except:
        pass

# 1. 크롬 옵션
options = webdriver.ChromeOptions()
options.add_experimental_option("prefs", {
    "download.default_directory": 로컬경로,
    "download.prompt_for_download": False,
    "safebrowsing.enabled": True
})

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
wait = WebDriverWait(driver, 20)

# 2. 로그인
driver.get("https://data.kigam.re.kr/auth/login")

wait.until(EC.visibility_of_element_located((By.NAME, "uid"))).send_keys(아이디)
driver.find_element(By.NAME, "pwd").send_keys(비밀번호)
driver.find_element(By.CLASS_NAME, "loginBtn").click()

wait.until(EC.url_changes("https://data.kigam.re.kr/auth/login"))
print("로그인 성공")

# 3. 연속파 페이지
driver.get("https://data.kigam.re.kr/quake/data/continuous-data")
wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
time.sleep(3)

# 4. 관측소: CHS(청송) 선택

station_mode_label = wait.until(
    EC.element_to_be_clickable(
        (By.XPATH, "//label[.//span[text()='직접 선택']]")
    )
)

driver.execute_script("""
    arguments[0].scrollIntoView({block: 'center'});
    arguments[0].click();
""", station_mode_label)

chs_item = wait.until(
    EC.element_to_be_clickable(
        (By.XPATH, "//li[@title='CHS']")
    )
)

driver.execute_script("""
    arguments[0].scrollIntoView({block: 'center'});
    arguments[0].click();
""", chs_item)

print("관측소 CHS(청송) 선택 완료")
time.sleep(1)


# 5. 채널: 직접 입력 활성화

channel_mode_label = wait.until(
    EC.element_to_be_clickable(
        (By.XPATH, "//label[@for='channelFilter.selectorType.byCodes']")
    )
)

driver.execute_script("""
    arguments[0].scrollIntoView({block: 'center'});
    arguments[0].click();
""", channel_mode_label)

time.sleep(0.5)

channel_input = wait.until(
    EC.visibility_of_element_located(
        (By.NAME, "channelCodes")
    )
)

channel_input.clear()
channel_input.send_keys("HHE,HHN,HHZ")

print("채널 HHE, HHN, HHZ 입력 완료")


# 6. 활용 목적: 연구(공학)

purpose_label = wait.until(
    EC.element_to_be_clickable(
        (By.XPATH, "//label[.//span[text()='연구(공학)']]")
    )
)

driver.execute_script("""
    arguments[0].scrollIntoView({block: 'center'});
    arguments[0].click();
""", purpose_label)

print("활용 목적: 연구(공학) 선택 완료")
time.sleep(0.5)


# 7. 날짜 루프 (2017년, 일 단위)

start_date = datetime(2017, 1, 1)
end_date = datetime(2017, 12, 31)
current = start_date

def set_datetime(name, value):
    handle_alert()
    el = wait.until(EC.presence_of_element_located((By.NAME, name)))

    driver.execute_script("""
        arguments[0].scrollIntoView({block: 'center'});
        arguments[0].focus();
        arguments[0].click();
    """, el)

    time.sleep(0.1)
    el.send_keys(Keys.CONTROL, "a")
    el.send_keys(Keys.DELETE)
    el.send_keys(value)
    el.send_keys(Keys.TAB)
    time.sleep(0.2)

# KST 설정
try:
    kst_radio = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable(
            (By.XPATH, "//span[contains(text(),'KST')]/preceding-sibling::input")
        )
    )
    driver.execute_script("arguments[0].click();", kst_radio)
    print("KST 설정 완료")
except:
    print("KST 이미 설정됨")

while current <= end_date:
    s = current.strftime("%Y-%m-%d 00:00:00")
    e = current.strftime("%Y-%m-%d 23:59:59")

    set_datetime("startTime", s)
    set_datetime("endTime", e)

    time.sleep(0.5)
    handle_alert()

    download_btn = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(text(),'다운로드')]")
        )
    )
    driver.execute_script("""
        arguments[0].scrollIntoView({block: 'center'});
        arguments[0].click();
    """, download_btn)

    handle_alert()
    print(f"요청 완료: {s}")

    time.sleep(12)
    current += timedelta(days=1)

print("2017년 CHS(청송) 전체 다운로드 요청 완료")


✅ 로그인 성공
✅ 관측소 CHS(청송) 선택 완료
✅ 채널 HHE, HHN, HHZ 입력 완료
✅ 활용 목적: 연구(공학) 선택 완료
ℹ️ KST 이미 설정됨
📥 요청 완료: 2017-01-01 00:00:00
📥 요청 완료: 2017-01-02 00:00:00
📥 요청 완료: 2017-01-03 00:00:00
📥 요청 완료: 2017-01-04 00:00:00
📥 요청 완료: 2017-01-05 00:00:00
📥 요청 완료: 2017-01-06 00:00:00
📥 요청 완료: 2017-01-07 00:00:00
📥 요청 완료: 2017-01-08 00:00:00
📥 요청 완료: 2017-01-09 00:00:00
📥 요청 완료: 2017-01-10 00:00:00
📥 요청 완료: 2017-01-11 00:00:00
📥 요청 완료: 2017-01-12 00:00:00
📥 요청 완료: 2017-01-13 00:00:00
📥 요청 완료: 2017-01-14 00:00:00
📥 요청 완료: 2017-01-15 00:00:00
📥 요청 완료: 2017-01-16 00:00:00
📥 요청 완료: 2017-01-17 00:00:00
📥 요청 완료: 2017-01-18 00:00:00
📥 요청 완료: 2017-01-19 00:00:00
📥 요청 완료: 2017-01-20 00:00:00
📥 요청 완료: 2017-01-21 00:00:00
📥 요청 완료: 2017-01-22 00:00:00
📥 요청 완료: 2017-01-23 00:00:00
📥 요청 완료: 2017-01-24 00:00:00
📥 요청 완료: 2017-01-25 00:00:00
📥 요청 완료: 2017-01-26 00:00:00
📥 요청 완료: 2017-01-27 00:00:00
📥 요청 완료: 2017-01-28 00:00:00
📥 요청 완료: 2017-01-29 00:00:00
📥 요청 완료: 2017-01-30 00:00:00
📥 요청 완료: 2017-01-31 00:00:00
📥 요청 완료: 20